In [2]:
import torch
import sys
import torch
import os.path as osp
import tqdm
import sys
import numpy as np
sys.path.append('')
from src.dataset.dataset import SimpleIterDataset
from src.utils.utils import to_filelist
from torch.utils.data import DataLoader
import dgl
import uproot
import plotly
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib.patches import FancyArrowPatch
import networkx as nx

In [3]:
class Args:
    def __init__(self):
        self.data_train = ['']
        self.data_val = ['']
        #self.data_train = files_train
        self.data_config = ''
        self.extra_selection = None
        self.train_val_split = 0.8
        self.data_fraction = 1
        self.file_fraction = 1
        self.fetch_by_files = False
        self.fetch_step = 0.01
        self.steps_per_epoch = None
        self.in_memory = False
        self.local_rank = None
        self.copy_inputs = False
        self.no_remake_weights = False
        self.batch_size = 10
        self.num_workers = 0
        self.demo = False
        self.laplace = False
        self.diffs = False
        self.class_edges = False
args = Args()
train_range = (0, args.train_val_split)
train_file_dict, train_files = to_filelist(args, 'train')
train_data = SimpleIterDataset(train_file_dict, args.data_config, for_training=True,
                                   extra_selection=args.extra_selection,
                                   remake_weights= True,
                                   load_range_and_fraction=(train_range, args.data_fraction),
                                   file_fraction=args.file_fraction,
                                   fetch_by_files=args.fetch_by_files,
                                   fetch_step=args.fetch_step,
                                   infinity_mode= False,
                                   in_memory=args.in_memory,
                                   async_load = False,
                                   name='train')
iterator = iter(train_data)
#g,gt = next(iterator)
#g,gt

=== Restarting DataIter train, seed=None ===


In [6]:
number_of_plots = 10 
for i in range(number_of_plots):
    g,gt = next(iterator)
    list_graph = dgl.unbatch(g)
    number_of_iteration = list_graph[0]
    particle_id = number_of_iteration.ndata['particle_number']
    hit_type = number_of_iteration.ndata['hit_type']
    pos = number_of_iteration.ndata['pos_hits_norm']
    pos_particles = gt[:,0:3]
    e_particle = gt[:,3]
    #change symbol by hit_type 
    marker_symbols = ['circle', 'circle-open', 'cross', 'diamond']
    max_hit_type = hit_type.argmax(1)
    ms = [marker_symbols[i.item()] for i in max_hit_type]
    #10 plots particles and hits 
    fig = make_subplots(rows= 10, cols= 1)
    #fig plot particles with size of particle energy color particle_id, shape hit_type
    fig.add_trace(go.Scatter3d( x= pos[:,0], y= pos[:,1], z=pos[:,2], mode= 'markers', marker= dict(size = e_particle, color = particle_id, 
    symbol = ms)))
    #fig plot hits with size of particle energy color particle_id, shape hit_type
    fig.add_trace(go.Scatter3d(x= pos_particles[:,0], y= pos_particles[:,1], z= pos_particles[:,2], mode = "markers", marker = dict(size = e_particle,color = particle_id,
     symbol = ms)))
    #update plot size 
    fig.update_layout(
                  height=500,  
                  width=500,  
                  showlegend=True,
                  autosize =True, 
                  #margin = dict( t= 0, b= 0, l=0, r= 0),
                  template= "plotly_white")
    #3D scene options
    fig.update_scenes(
        aspectratio =dict(x=1 , y=1, z= 1),
        aspectmode = "manual")
    #dropdown menu 
    button_layer_1_height = 1
    fig.update_layout(
        updatemenus = [
            dict(
                buttons = list([
                    dict(label = "Plots",
                         method = "update",
                         args = [{"visible": [True, True]},
                                 {"title": "Plots"}]),
                    dict(label = "particle_id",
                    method = "update", 
                    args = [{"visible": [True ,False]},
                    {"title": "particles"}]),
                    dict (label = "hits",
                    method = "update",
                    args = [{"visible" : [False , True]},
                    {"title": "hits"}])
                    ])
                    
            )
        ]
    )
    fig.update_layout()
    fig.show()
    #pyo.plot(fig, filename='plot1.html')
    #pyo.iplot(fig)